In [25]:
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.externals import joblib
import numpy as np
import sys
sys.path.append('../../../code/utils/')
import perf_utils as pu

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
torch.manual_seed(20180429)    # reproducible
# Hyper Parameters                                      #Haven't figured out, let these just be here
BATCH_SIZE = 512

In [6]:
def csr_to_tensor(csr_matrix, size):
    data = csr_matrix.data
    indices = csr_matrix.indices
    
    # http://pytorch.org/docs/stable/sparse.html
    i = torch.LongTensor([[0, num] for num in indices], device=device)
    v = torch.FloatTensor(data.astype(np.float), device=device)
    result_tensor = torch.sparse.FloatTensor(i.t(), v, size, device=device).to_dense()
    return result_tensor
        

In [7]:
# Here you should convert data into formats that torch accepts
# Make a dataloader
class MyDataset(Data.Dataset):
    def __init__(self, x, y):
        self.input = x
        self.label = y

    def __getitem__(self, index):#返回的是tensor
        # convert self.input[index] to tensor
        input_item = self.input[index]
        x = csr_to_tensor(input_item, torch.Size([1,self.input.shape[1]]))
        
        # convert self.label[index] to tensor
        y = torch.tensor(self.label[index])
        return x, y

    def __len__(self):
        return len(self.label)

In [8]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 32),
        ).to(device)
        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.Tanh(),
            nn.Linear(128, input_size),
        ).to(device)

    def forward(self, x):
        encoded = self.encoder(x.to(device))
        decoded = self.decoder(encoded)
        return encoded, decoded

In [9]:
ae = torch.load('./useall_step16000_ae_full_model_checkpoint.pth')

In [10]:
train_X = joblib.load('../../../../../zhangez698/TencentAlgo2018/playground/Elvin/autoencoder/xxx.pkl') # on Server
train_y = joblib.load('../../../../../zhangez698/TencentAlgo2018/playground/Elvin/autoencoder/yyy.pkl')

In [11]:
train_X.shape

(8798814, 419862)

In [17]:
BATCH_SIZE = 256

In [18]:
train_dataset = MyDataset(train_X, train_y)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
results = []
for step, (x, y) in enumerate(train_loader): # step: batch index; x.shape: [BATCH_SIZE, 1, 374251]; y.shape: [BATCH_SIZE]   
    b_x = torch.tensor(x).to(device)                   # batch x
    b_y = torch.tensor(x).to(device)
    b_label = torch.tensor(y).to(device)               # batch label

    encoded, _ = ae(b_x)
    results.append(encoded)
    if step % 300 == 0:
        print('Step {}, Current time: {}'.format(step, pu.get_time_str()))
joblib.dump(results, './encoded_data.pkl')

Step 0, Current time: 14:03:12


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58